In [1]:
import numpy as np
from ppopt.mplp_program import MPLP_Program
from ppopt.mpmodel import MPModeler
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
from ppopt.plot import parametric_plot
from numpy.polynomial.legendre import leggauss
from scipy.optimize import linprog
from typing import Union
from collections import defaultdict
from itertools import product
import time
import sympy as sp
from typing import Union

In [2]:
# for i in range(len(solution_flexibility)):
#     print(f'i:{i}\nA:{solution_flexibility.critical_regions[i].A}\nb:{solution_flexibility.critical_regions[i].b}')

In [3]:
def gauss_legendre_between_bounds(expr_coeffs: np.ndarray, n_gl: int, max_idx: int = 0, min_idx: int = 1):
    """
    Generate n Gauss–Legendre quadrature points and weights between min and max bounds
    defined by two linear expressions.

    Parameters:
        expr_coeffs (np.ndarray): 2xD array. Row 0 = max point co`efficients, Row 1 = min.
        n (int): Number of quadrature points.

    Returns:
        points (np.ndarray): (n, D) array of quadrature points.
        weights (np.ndarray): (n,) array of weights.
    """
    if expr_coeffs.shape[0] != 2:
        raise ValueError("expr_coeffs must have two rows")

    max_coeffs = expr_coeffs[max_idx]
    min_coeffs = expr_coeffs[min_idx]

    # Get Gauss–Legendre points and weights on [-1, 1]
    nodes, weights = leggauss(n_gl)
    weights = weights.reshape(-1,1)
    
    # Affine transformation to domain [min_coeffs, max_coeffs]
    points = 0.5 * (np.outer((nodes + 1), max_coeffs) + np.outer((1 - nodes), min_coeffs))

    # Adjust weights to match new domain
    weights = 0.5 * weights@(max_coeffs - min_coeffs).reshape(1,-1)

    return points, weights

In [4]:
def get_quadrature_points(solution, nq: int, t_vector: np.ndarray):
    # Augment t_vector once
    t_vector_aug = np.append(t_vector, 1).reshape(-1, 1)

    if isinstance(solution, list):
        qpoints, qweights = np.polynomial.legendre.leggauss(nq)
        min, max = solution[0], solution[1]
        qps_mapped = 0.5*(max*(1+qpoints) + min*(1-qpoints))
        qws_mapped = 0.5*(max-min)*qweights
        # print(max, min, qps_mapped, qws_mapped)
        return max, min, qps_mapped, qws_mapped

    for region in solution.critical_regions:
        if region.is_inside(t_vector):
            coeffs = np.concatenate([region.A, region.b], axis=1)[:2, :]
            qpoints, qweights = gauss_legendre_between_bounds(expr_coeffs=coeffs, n_gl=nq)
            return coeffs[0] @ t_vector_aug, coeffs[1] @ t_vector_aug, qpoints @ t_vector_aug, qweights @ t_vector_aug

    # print(f't_vector: {t_vector}')
    # print(f'solution:{solution}')
    raise ValueError("No region found that contains the given t_vector.")


In [5]:
# def get_quadrature_points(solution, nq: int, t_vector: Union[np.ndarray, list]):
#     """
#     Returns symbolic coefficient matrices (not numerical evaluations).
#     Each row of t_points and t_weights is a list of coefficients for a linear expression in design variables.
#     Assumes t_vector is a placeholder and only used to determine dimensionality.
#     """
#     
#     t_vector_np = np.array(t_vector)
# 
#     if isinstance(solution, list):  # if bounds are given directly
#         qpoints_1d, qweights_1d = np.polynomial.legendre.leggauss(nq)
#         min_val, max_val = solution[0], solution[1]
# 
#         # Map points and weights from [-1,1] to [min, max]
#         qps_mapped = 0.5 * (max_val * (1 + qpoints_1d) + min_val * (1 - qpoints_1d))
#         qws_mapped = 0.5 * (max_val - min_val) * qweights_1d
# 
#         # Represent each point and weight as a constant + 0*d1 + 0*d2 + ...
#         d_len = len(t_vector) if hasattr(t_vector, '__len__') else 1
#         coeffs_shape = (nq, d_len + 1)  # +1 for constant
#         t_point_coeffs = np.zeros(coeffs_shape)
#         t_weight_coeffs = np.zeros(coeffs_shape)
# 
#         t_point_coeffs[:, -1] = qps_mapped
#         t_weight_coeffs[:, -1] = qws_mapped
# 
#         return max_val, min_val, t_point_coeffs, t_weight_coeffs
# 
#     # Region-based evaluation
#     for region in solution.critical_regions:
#         if region.is_inside(t_vector_np):
#             # expr_coeffs: 2 rows, each row is coefficients for [d1, d2, ..., 1]
#             expr_coeffs = np.concatenate([region.A, region.b], axis=1)[:2, :]
# 
#             qpoints, qweights = gauss_legendre_between_bounds(expr_coeffs=expr_coeffs, n_gl=nq)
# 
#             # Return coefficient matrices directly instead of evaluating at t_vector
#             return (
#                 expr_coeffs[0], expr_coeffs[1],  # max, min coeffs (can be ignored if unused)
#                 qpoints, qweights                # each row: [a1, a2, ..., constant]
#             )
# 
#     raise ValueError("No region found that contains the given t_vector.")


In [6]:
def mpformulate_theta_bounds(flex_sol, num_theta:int, theta_bounds:list, num_design:int=0, design_bounds:list=None, psi_idx:int=0, theta_m:int=0):
    A0, b0, F0 = np.empty((len(flex_sol), num_theta)), np.empty((len(flex_sol), 1)), np.empty((len(flex_sol), num_design))
    num_cr = len(flex_sol.critical_regions)
    for i, region in enumerate(flex_sol.critical_regions):
        A0[i] = region.A[psi_idx,:num_theta]
        b0[i] = -region.b[psi_idx]
        F0[i] = -region.A[psi_idx, num_theta:num_theta+num_design]
    # print(f'num_cr:{num_cr}')
    # print(f'num_theta:{num_theta}')
    # print(f'num_design:{num_design}')
    # print(f"A0: {A0}")
    # print(f"b0: {b0}")
    # print(f"F0: {F0}")
    
    c = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))]).reshape(-1,1)
    # print(f'c:{c}')
    # print(f'c.shape: {c.shape}')
    
    row1_block = np.hstack([block for i in range(theta_m, num_theta) for block in (A0[:, [i]], np.zeros((num_cr, 1)))])
    row2_block = np.hstack([block for i in range(theta_m, num_theta) for block in (np.zeros((num_cr, 1)), A0[:, [i]])])
    bound_row = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))])
    A = np.vstack([row1_block, row2_block, bound_row, -np.eye(2*(num_theta-theta_m)), np.eye(2*(num_theta-theta_m))])
    # print(f'A: {A}')
    # print(f'A.shape: {A.shape}')
    
    x_lb = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][0]] * 2])
    x_ub = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][1]] * 2])
    b = np.vstack([b0, b0, np.zeros((1,1)), -x_lb.reshape(-1,1), x_ub.reshape(-1,1)])
    # print(f'b: {b}')
    # print(f'b.shape: {b.shape}')
    
    if F0.size==0 and theta_m==0:
        # print('here')
        return A, b, c, np.array([]), np.array([]), np.array([]), np.array([]) 
    
    F = np.vstack([F0, F0, np.zeros((1,num_design)), np.zeros((4*(num_theta-theta_m), num_design))]) if num_design>0 else np.vstack([F0, F0])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    if theta_m > 0:
        F_lltheta = np.hstack([A0[:, [i]] for i in range(theta_m)])
        # print(f'F_lltheta: {F_lltheta}')
        # print(f'F_lltheta.shape: {F_lltheta.shape}')
        F = np.hstack([np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))]), F]) if F.size > 0 else np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    
    H = np.zeros((2*(num_theta-theta_m), theta_m+num_design))
    # print(f'H:{H}')
    # print(f'H.shape: {H.shape}')
    
    A_t = np.vstack([-np.eye(theta_m+num_design), np.eye(theta_m+num_design)])
    # print(f'A_t:{A_t}')
    # print(f'A_t.shape: {A_t.shape}')
    
    theta_lb = np.array([-theta_bounds[i][0] for i in range(theta_m)] + ([-j[0] for j in design_bounds] if isinstance(design_bounds, list) 
                                                                        else [])).reshape(-1, 1)
    theta_ub = np.array([theta_bounds[i][1] for i in range(theta_m)] + ([j[1] for j in design_bounds] if isinstance(design_bounds, list) 
                                                                        else [])).reshape(-1, 1)
    
    b_t = np.vstack([theta_lb, theta_ub])
    # print(f'b_t:{b_t}')
    # print(f'b_t.shape: {b_t.shape}')
    
    return A, b, c, H, A_t, b_t, F

In [7]:
def get_theta_bounds(flex_sol, numt, tbounds, numd:int=0, dbounds:list=None):
    
    theta_bound_dict = defaultdict(dict)
    prob_dict = defaultdict(dict)
    for i in range(numt):
        A, b, c, H, A_t, b_t, F = mpformulate_theta_bounds(flex_sol=flex_sol, num_theta=numt ,num_design=numd, theta_bounds=tbounds, design_bounds=dbounds, theta_m=i)
        if F.size != 0:
            prob = MPLP_Program(A=A, b=b, c=c, H=H, A_t=A_t, b_t=b_t, F=F)
            prob.process_constraints()
            solution = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.combinatorial)
            prob_dict[f't{i}'] = prob
            theta_bound_dict[f't{i}'] = solution
        else:
            linsol = linprog(c=c, A_ub=A, b_ub=b)
            prob_dict[f't{i}'] = linsol
            theta_bound_dict[f't{i}'] = [linsol.x[1], linsol.x[0]]
            # if linsol.success:
                # print("Optimal value:", linsol.fun)
                # print("Optimal x:", linsol.x)
        print(f'Finished solving for theta{i+1}')
    probs = [p for key, p in prob_dict.items()]
    sols = [sol for key, sol in theta_bound_dict.items()]
    
    return probs, sols
    

In [8]:
def calculate_stocflexibility(sols, nq: Union[int, list], joint_func, d_vector: np.ndarray = None):
    
    # Validate nq if it's a list
    if isinstance(nq, list):
        if len(nq) != len(sols):
            raise ValueError("If nq is a list, it must have the same length as sols")

    def recurse(level: int, theta_prev: list, weight_prev: float) -> float:
        """
        Recursive inner function to compute nested quadrature.
        """
        # print(f'level:{level}')
        if level == len(sols):
            return weight_prev * joint_func(theta_prev)

        # Use nq[level] if nq is a list, otherwise use scalar nq
        nql = nq[level] if isinstance(nq, list) else nq

        t_vector = np.block([np.array(theta_prev), d_vector]) if isinstance(d_vector, np.ndarray) else np.array(theta_prev)
        # print(f't_vector:{t_vector}')
        # print(f'probs[{level}].A:{probs[level].A}')
        _, _, t_points, t_weights = get_quadrature_points(solution=sols[level], nq=nql, t_vector=t_vector)

        t_points = t_points.flatten()
        t_weights = t_weights.flatten()
        # print('t_points:', t_points)
        # print(f'theta_prev: {theta_prev}')
        return sum(recurse(level + 1, theta_prev + [v], weight_prev * w) for v, w in zip(t_points, t_weights))
    s = time.time()
    stflex =  recurse(level=0, theta_prev=[], weight_prev=1.0)
    e = time.time()
    print(f'Elapsed time for calculating sf index: {e- s}')
    
    return stflex
    

In [9]:
# Bansal (2000) Illustrative Example
t_bounds=[(0,4),(0,4)]
d_bounds=[(0,5), (0,5)]
nt = len(t_bounds)
nd = len(d_bounds)

m = MPModeler()

u = m.add_var(name='u')
x = m.add_var(name='x')
z = m.add_var(name='z')

t1 = m.add_param(name='t1')
t2 = m.add_param(name='t2')
d1 = m.add_param(name='d1')
d2 = m.add_param(name='d2')
m.add_constr(2*x - 3*z + t1 - d2 == 0)
m.add_constr(x - z/2 -t1/2 +t2/2 +d1 -7*d2/2 <= u)
m.add_constr(-2*x +2*z -4*t1/3 -t2 +2*d2 +1/3<= u)
m.add_constr(-x + 5*z/2 +t1/2 -t2 -d1 +d2/2 -1 <= u)
m.add_constr(-50 <= x)
m.add_constr(-50 <= z)
m.add_constr(t_bounds[0][0] <= t1)
m.add_constr(t_bounds[1][0] <= t2)
m.add_constr(d_bounds[0][0] <= d1)
m.add_constr(d_bounds[1][0] <= d2)
m.add_constr(t1 <= t_bounds[0][1])
m.add_constr(t2 <= t_bounds[1][1])
m.add_constr(d1 <= d_bounds[0][1])
m.add_constr(d2 <= d_bounds[1][1])
m.set_objective(u)
prob = m.formulate_problem()
prob.process_constraints()
solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)

start_time = time.time()
prob_list, sol_list = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)
end_time = time.time()
print(f'Elapsed time for solving mp problems: {end_time-start_time}')

def joint_pdf(theta:list):
    return (2/np.pi)*np.exp(-2*((theta[0]-2)**2 + (theta[1]-2)**2))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-20
Using a found active set [0, 1, 2]
Finished solving for theta1
Finished solving for theta2
Elapsed time for solving mp problems: 0.4784696102142334


In [30]:
d_vector = np.array([4, 0.5])
nq = 5
sf_idx = calculate_stocflexibility(sols=sol_list, nq=nq, joint_func=joint_pdf, d_vector=d_vector)
print(f'Stochastic Flexibility Index: {sf_idx:.4}')

Elapsed time for calculating sf index: 0.0020580291748046875
Stochastic Flexibility Index: 0.6372


In [11]:
# # Straub and Grossmann (1990) Example 2
# t_bounds=[(0,80),(0,80)]
# d_bounds=[(0,10), (0,10)]
# nt = len(t_bounds)
# nd = len(d_bounds)
# m = MPModeler()
# 
# u = m.add_var(name='u')
# z = m.add_var(name='z')
# 
# t1 = m.add_param(name='t1')
# t2 = m.add_param(name='t2')
# d1 = m.add_param(name='d1')
# d2 = m.add_param(name='d2')
# m.add_constr(z - 1.6*(t1-20) - 0.6*(t2-20) + d1 - 14 <= u)
# m.add_constr(z - 0.85*(t1-20) - 0.925*(t2-20) + d1 + d2 -20 <= u)
# m.add_constr(z - 1.1*(t1-20) - 1.4*(t2-20) + d2 - 8 <= u)
# m.add_constr(-z + (t1-20) + (t2-20) + d1 + d2 - 7 <= u)
# m.add_constr(t_bounds[0][0] <= t1)
# m.add_constr(t_bounds[1][0] <= t2)
# m.add_constr(d_bounds[0][0] <= d1)
# m.add_constr(d_bounds[1][0] <= d2)
# m.add_constr(t1 <= t_bounds[0][1])
# m.add_constr(t2 <= t_bounds[1][1])
# m.add_constr(d1 <= d_bounds[0][1])
# m.add_constr(d2 <= d_bounds[1][1])
# m.set_objective(u)
# prob = m.formulate_problem()
# prob.process_constraints()
# solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
# 
# start_time = time.time()
# prob_list, sol_list = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)
# end_time = time.time()
# print(f'Elapsed time for solving mp problems: {end_time-start_time}')
# 
# def joint_pdf(theta:list):
#     return (1/628.32)*np.exp((-(theta[0]-40)**2 - (theta[1]-40)**2)/200)

In [12]:
# d_vector = np.array([1,8])
# nq = 7
# sf_idx = calculate_stocflexibility(sols=sol_list, nq=nq, joint_func=joint_pdf, d_vector=d_vector)
# print(f'Stochastic Flexibility Index: {sf_idx:.4}')

In [13]:
# # Pistikopoulos and Mazzuchi (1990) Example 1
# t_bounds = [(610, 630), (378, 398), (573, 593), (303, 323)]
# # t_bounds = [(586.68, 653.32), (354.68, 421.32), (549.68, 616.32), (279.68, 346.32)]
# nt = len(t_bounds)
# nd = 0
# d_bounds = None
# m = MPModeler()
# 
# u = m.add_var(name="u")
# z = m.add_var(name="z")
# 
# t1 = m.add_param(name="t1")
# t2 = m.add_param(name="t2")
# t3 = m.add_param(name="t3")
# t4 = m.add_param(name="t4")
# 
# m.add_constr(-z + 1.5*t2 - 525 <= u)
# m.add_constr(z - 1.5*t1 - 2*t2 - 2*t3 + 2777 <= u)
# m.add_constr(z - 1.5*t1 - 2*t2 - t3 + 2044 <= u)
# m.add_constr(z - 1.5*t1 - 2*t2 - t3 - 2*t4 + 2830 <= u)
# m.add_constr(-z + 1.5*t1 + 2*t2 + t3 + 3*t4 - 3153 <= u)
# m.add_constr(t_bounds[0][0] <= t1)
# m.add_constr(t1 <= t_bounds[0][1])
# m.add_constr(t_bounds[1][0] <= t2)
# m.add_constr(t2 <= t_bounds[1][1])
# m.add_constr(t_bounds[2][0] <= t3)
# m.add_constr(t3 <= t_bounds[2][1])
# m.add_constr(t_bounds[3][0] <= t4)
# m.add_constr(t4 <= t_bounds[3][1])
# m.set_objective(u)
# prob = m.formulate_problem()
# prob.process_constraints()
# 
# solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
# 
# start_time = time.time()
# prob_list, sol_list = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)
# end_time = time.time()
# print(f'Elapsed time for solving mp problems: {end_time-start_time}')
# 
# def joint_pdf(theta:list):
#     return (1/(5000*np.pi))*np.exp(-0.08*((theta[0]-620)**2 + (theta[1]-388)**2))

In [14]:
# d_vector = None
# nq = 8
# sf_idx = calculate_stocflexibility(sols=sol_list, nq=nq, joint_func=joint_pdf, d_vector=d_vector)
# print(f'Stochastic Flexibility Index: {sf_idx:.4}')

In [15]:
# # Bansal (2000) Example 4
# t_bounds = [(390, 410), (440, 460), (390, 410), (420, 440), (300, 320), (280, 300), (275, 295)]
# nt = len(t_bounds)
# nd = 0
# d_bounds = None
# m = MPModeler()
# 
# u = m.add_var('u')
# z = m.add_var('z')
# 
# t1 = m.add_param('t1')
# t2 = m.add_param('t2')
# t3 = m.add_param('t3')
# t4 = m.add_param('t4')
# t5 = m.add_param('t5')
# t6 = m.add_param('t6')
# t7 = m.add_param('t7')
# 
# m.add_constr(5/12 * z + 1/3 * t3 + t6 + 1/3 * t7 + -(682 + 1/2) <= u)
# m.add_constr(5/12 * z - 1/3 * t1 - 1/6 * t2 + 1/3 * t3 - 5/12 * t5 + t6 + 1/3 * t7 - (357 + 1/2) <= u)
# m.add_constr(-7/12 * z + 2/3 * t1 + 1/3 * t2 + 1/3 * t3 + 5/6 * t5 + t6 + 1/3 * t7 - (1007 + 1/2) <= u)
# m.add_constr(-5/12 * z - 1/3 * t3 - 1/3 * t7 + (357 + 1/2) <= u)
# m.add_constr(z - t4 <= u)
# m.add_constr(4/5 * t3 + t6 + 4/5 * t7 - 858 <= u)
# m.add_constr(t3 + t7 - 700 <= u)
# 
# m.add_constr(t_bounds[0][0] <= t1)
# m.add_constr(t1 <= t_bounds[0][1])
# m.add_constr(t_bounds[1][0] <= t2)
# m.add_constr(t2 <= t_bounds[1][1])
# m.add_constr(t_bounds[2][0] <= t3)
# m.add_constr(t3 <= t_bounds[2][1])
# m.add_constr(t_bounds[3][0] <= t4)
# m.add_constr(t4 <= t_bounds[3][1])
# m.add_constr(t_bounds[4][0] <= t5)
# m.add_constr(t5 <= t_bounds[4][1])
# m.add_constr(t_bounds[5][0] <= t6)
# m.add_constr(t6 <= t_bounds[5][1])
# m.add_constr(t_bounds[6][0] <= t7)
# m.add_constr(t7 <= t_bounds[6][1])
# m.set_objective(u)
# prob = m.formulate_problem()
# prob.process_constraints()
# 
# solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
# 
# start_time = time.time()
# prob_list, sol_list = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)
# end_time = time.time()
# print(f'Elapsed time for solving mp problems: {end_time-start_time}')
# 
# def joint_pdf(theta:list):
#     return (2.5e-9) * (0.05*(theta[0]-390))**(0.2) * (1 - (0.05*(theta[0]-390)))**(0.2) * (0.05*(theta[1]-440))**(0.2) * (1 - (0.05*(theta[1]-440)))**(0.2) * (0.05*(theta[4]-300))**(0.2) * (1 - (0.05*(theta[4]-300)))**(0.2)

In [16]:
# d_vector = None
# nq = 12
# sf_idx = calculate_stocflexibility(sols=sol_list, nq=nq, joint_func=joint_pdf, d_vector=d_vector)
# print(f'Stochastic Flexibility Index: {sf_idx:.4}')

In [17]:
# def get_stocflexibility(flexibility_sol, nt:int, nq: Union[int, list], theta_bounds:list, nd:int=0, d_vector: np.ndarray = None, design_bounds:list = None) -> float:
#     """
#     Calculates the stochastic flexibility index via nested Gauss–Legendre quadrature.
# 
#     :param sols: list of solution objects (one per stage)
#     :param nq: int or list of ints; if int, used for all levels; if list, should match len(sols)
#     :param d_vector: base vector used to condition quadrature
#     :return: stochastic flexibility index
#     """
#     start_time = time.time()
#     probs, sols = get_theta_bounds(flex_sol=flexibility_sol, numt=nt, numd=nd, tbounds=theta_bounds, dbounds=design_bounds)
#     end_time = time.time()
#     print(f'Elapsed time for solving mp problems: {end_time-start_time}')
#     
#     stflex = calculate_stocflexibility(sols=sols, nq=nq, d_vector=d_vector)
#     
#     return probs, sols, stflex

# prob_list, sol_list, sf_idx = get_stocflexibility(flexibility_sol=solution_flexibility, nt=nt, nq=16, theta_bounds=t_bounds, nd=nd, design_bounds=d_bounds, d_vector=d_vector)
# start_time = time.time()
# prob_list, sol_list = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)
# end_time = time.time()
# print(f'Elapsed time for solving mp problems: {end_time-start_time}')
# 
# nq = 12
# sf_idx = calculate_stocflexibility(sols=sol_list, nq=nq, joint_func=joint_pdf, d_vector=d_vector)
# print(f'Stochastic Flexibility Index: {sf_idx:.4}')

In [18]:
# data_4_expr = defaultdict(dict)
# for t, sol in enumerate(sol_list):
#     if isinstance(sol, list):
#         data_4_expr[t] = {'max': np.array([sol[1]]),
#                           'min': np.array([sol[0]]),
#                           'is_mp': False, 'E': True, 'F': True}
#     else:
#         for cr, region in enumerate(sol.critical_regions):
#             data_4_expr[t][cr] = {'max': np.concatenate([region.A, region.b], axis=1)[0, :],
#                                   'min': np.concatenate([region.A, region.b], axis=1)[1, :],
#                                   'is_mp': True, 'E': region.E, 'f': region.f}

In [19]:
# data_4_expr[0]

In [20]:
# def calculate_symbolic_stocflexibility(flexibility_sol, nt:int, nq: Union[int, list], design_vars: list, theta_bounds:list, nd:int=0, design_bounds:list = None) -> sp.Expr:
#     """
#     Symbolically calculates the stochastic flexibility index using Gauss–Legendre quadrature,
#     where quadrature points and weights are linear expressions in design variables.
# 
#     :param sols: list of solution objects
#     :param nq: number of quadrature points (int or list of ints)
#     :param design_vars: list of sympy symbols representing design variables (e.g. [d1, d2, ...])
#     :return: sympy expression representing stochastic flexibility index
#     """
#     
#     start_time = time.time()
#     probs, sols = get_theta_bounds(flex_sol=flexibility_sol, numt=nt, numd=nd, tbounds=theta_bounds, dbounds=design_bounds)
#     end_time = time.time()
#     print(f'Elapsed time for solving mp problems: {end_time-start_time}')
#     
#     if isinstance(nq, list):
#         if len(nq) != len(sols):
#             raise ValueError("If nq is a list, it must have the same length as sols")
#     
#     def recurse(level: int, theta_prev: list, weight_prev: sp.Expr) -> sp.Expr:
#         if level == len(sols):
#             return weight_prev * joint_pdf(theta_prev)
# 
#         # Resolve nq for this level
#         nql = nq[level] if isinstance(nq, list) else nq
# 
#         # Assume get_quadrature_points returns symbolic coefficient arrays of shape (nq, len(design_vars)+1)
#         _, _, t_point_coeffs, t_weight_coeffs = get_quadrature_points(solution=sols[level], nq=nql, t_vector=design_vars)
# 
#         expr = 0
#         for i in range(t_point_coeffs.shape[0]):
#             # Construct θᵢ and wᵢ as symbolic expressions
#             θ_i = sum(t_point_coeffs[i, j] * design_vars[j] for j in range(len(design_vars))) + t_point_coeffs[i, -1]
#             w_i = sum(t_weight_coeffs[i, j] * design_vars[j] for j in range(len(design_vars))) + t_weight_coeffs[i, -1]
# 
#             expr += recurse(level + 1, theta_prev + [θ_i], weight_prev * w_i)
# 
#         return expr
#     s = time.time()
#     stflex_expr =  recurse(0, [], 1)
#     e = time.time()
#     print(f'Elapsed time for calculating sf index: {e - s}')
#     return probs, sols, stflex_expr

In [21]:
# prob_list, sol_list, sf_expre = calculate_symbolic_stocflexibility(flexibility_sol=solution_flexibility, nt=nt, nq=8, theta_bounds=t_bounds, nd=nd, design_bounds=d_bounds, design_vars=sp.symbols('d1 d2 d3'))